## Кластерный анализ методом DBSCAN

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [2]:
df = pd.read_csv('/Users/d.kapanadze/Downloads/Учеба/ИТМО/1 семестр/Анализ данных на Python/Задачи на первую лабу/экономика городов/Econom_Cities_data.csv', sep=';')

# Преобразуем к числовому типу
df['Price'] = df['Price'].str.replace(',', '.').astype(float)
df['Salary'] = df['Salary'].str.replace(',', '.').astype(float)



df.head()

,City,Work,Price,Salary
0,Amsterdam,1714,65.6,49.0
1,Athens,1792,53.8,30.4
2,Bogota,2152,37.9,11.5
3,Bombay,2052,30.3,5.3
4,Brussels,1708,73.8,50.5


In [5]:
# Удаляем категориальную переменную
df_num = df.drop('City', axis=1)

# Стандартизируем данные
scaler = StandardScaler()
df_stand = pd.DataFrame(scaler.fit_transform(df_num), columns=df_num.columns)

In [6]:
df_stand.head()

,Work,Price,Salary
0,0.138269,-0.151250,0.213212
1,0.171046,-0.698647,0.203941
2,0.322325,-1.436242,0.194519
3,0.280303,-1.788804,0.191429
4,0.135748,0.229145,0.213960


In [7]:
df_stand.shape

(48, 3)

In [8]:
# Первая кластеризация методом DBSCAN(параметры выбираем случайные)
clust = DBSCAN(eps=2, metric='euclidean', min_samples=4)

clust.fit(df_stand)

unique, counts = np.unique(clust.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)


[[-1  2]
 [ 0 46]]


Получили 2 выброса и 1 кластер из 46 элементов. Плохо. Судя по всему надо уменьшить n

In [10]:
clust = DBSCAN(eps=2, metric='euclidean', min_samples=2)

clust.fit(df_stand)

unique, counts = np.unique(clust.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)


[[ 0 46]
 [ 1  2]]


Теперь он не считает те две точки выбросами, но мы всё ещё имеем 1 большой кластер. Будем уменьшать эпсилон

In [14]:
clust = DBSCAN(eps=0.3, metric='euclidean', min_samples=2)

clust.fit(df_stand)

unique, counts = np.unique(clust.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)


[[-1  3]
 [ 0 37]
 [ 1  4]
 [ 2  4]]


In [15]:
clust = DBSCAN(eps=0.2, metric='euclidean', min_samples=2)

clust.fit(df_stand)

unique, counts = np.unique(clust.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)


[[-1  3]
 [ 0 37]
 [ 1  4]
 [ 2  4]]


In [13]:
clust = DBSCAN(eps=0.1, metric='euclidean', min_samples=2)

clust.fit(df_stand)

unique, counts = np.unique(clust.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)


[[-1 20]
 [ 0 11]
 [ 1  2]
 [ 2  2]
 [ 3  8]
 [ 4  5]]


In [16]:
clust = DBSCAN(eps=0.15, metric='euclidean', min_samples=2)

clust.fit(df_stand)

unique, counts = np.unique(clust.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)


[[-1  5]
 [ 0 21]
 [ 1 13]
 [ 2  2]
 [ 3  3]
 [ 4  4]]


Всё ещё не очень хороший результат(много маленьких кластеров). Попробуем поиск по сетке с оптимизацией `silhouette_score`

In [17]:
# Методом тыка понял что эпсилон больше 1 не имеет смысл брать
eps_grid = np.linspace(0.1, 1, 19).tolist()
n_grid = [2, 3, 4]

best_params = [0, 0]
best_score = 0 

for j in eps_grid:
    for i in n_grid:
        
        db = DBSCAN(eps=j, metric='euclidean', min_samples=i).fit(df_stand)

    if metrics.silhouette_score(df_stand, db.labels_) > best_score:
        best_params = [j, i]



In [18]:
best_params

[1.0, 4]

Судя по поиску, лучшие параметры: эпсилон = 1, n = 4. Посмотрим что даст такая кластеризация

In [19]:
clust = DBSCAN(eps=1, metric='euclidean', min_samples=4)

clust.fit(df_stand)

unique, counts = np.unique(clust.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)


[[-1  2]
 [ 0 46]]


"Лучшая" с точки зрения `silhouette_score` кластеризация оказалась бесполезной. Это можно было сказать сразу по значению эпсилон -- ранее поняли что эпсилон должно быть где-то в районе 0.1-0.2

Попробуем опять осуществить поиск по сетке, но уже с очень маленьким шагом по эпсилон

In [20]:
# Методом тыка понял что эпсилон больше 1 не имеет смысл брать
eps_grid = np.linspace(0.1, 0.2, 50).tolist()
n_grid = [2, 3, 4]

best_params = [0, 0]
best_score = 0 

for j in eps_grid:
    for i in n_grid:
        
        db = DBSCAN(eps=j, metric='euclidean', min_samples=i).fit(df_stand)

    if metrics.silhouette_score(df_stand, db.labels_) > best_score:
        best_params = [j, i]



In [21]:
best_params

[0.2, 4]

In [22]:
clust = DBSCAN(eps=0.2, metric='euclidean', min_samples=4)

clust.fit(df_stand)

unique, counts = np.unique(clust.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)


[[-1  8]
 [ 0 36]
 [ 1  4]]


Опять плохой результат. Добиться хорошей кластеризации не удалось